<a href="https://colab.research.google.com/github/tguerand/SentimentAnalysis/blob/master/bin/data_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import urllib.request
from sklearn.model_selection import train_test_split
print("Tensorflow version is :", tf.__version__)

Tensorflow version is : 2.3.0


#Access Path

In [2]:
!git clone https://github.com/tguerand/SentimentAnalysis
os.mkdir("./SentimentAnalysis/DATA")
#Download file
!gdown --id 1dO4aubOro159awBPzD675c22qtXMOVp0 --output ./SentimentAnalysis/DATA/training.csv

Cloning into 'SentimentAnalysis'...
remote: Enumerating objects: 71, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 71 (delta 24), reused 17 (delta 1), pack-reused 0
Unpacking objects: 100% (71/71), done.
Downloading...
From: https://drive.google.com/uc?id=1dO4aubOro159awBPzD675c22qtXMOVp0
To: /content/SentimentAnalysis/DATA/training.csv
239MB [00:02, 119MB/s]


In [ ]:
from SentimentAnalysis.bin import model 

#Import Data

In [3]:
df=pd.read_csv("./SentimentAnalysis/DATA/training.csv",encoding='latin-1')
df.columns=['target', 't_id', 'created_at', 'query', 'user', 'text']
#df.head()

In [4]:
print(df["text"].head(5).to_string())

0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 


#Data Analysis

#### Data preparation

In [5]:
#code Micka

df["nbhash"]=df["text"].str.count("#")>0
df["nbat"]=df["text"].str.count("@")>0
import re
df["text"] = df["text"].apply(lambda x : re.sub('@[a-zA-Z0-9]+', 'ààà', x))
df["text"] = df["text"].apply(lambda x : re.sub('http://[a-zA-Z0-9]+', 'urlofsite', x))
df["text"] = df["text"].apply(lambda x : re.sub('www.[a-zA-Z0-9]+', 'urlofsite', x))
df["text"] = df["text"].apply(lambda x : x.replace("!"," exclamationmark "))
df["text"] = df["text"].apply(lambda x : x.replace("?"," questionmark "))
df["text"] = df["text"].apply(lambda x : x.replace("&lt;3"," loveemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(":)"," smileemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(": )"," smileemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(":'("," cryemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(";)"," winkemoji "))
df["text"] = df["text"].apply(lambda x : x.replace(":/"," mehemoji "))
df["text"] = df["text"].apply(lambda x : x.replace("w/o"," without "))
df["text"] = df["text"].apply(lambda x : x.replace("b/c"," because "))
df["text"] = df["text"].apply(lambda x : x.replace(" w "," with "))
df["text"] = df["text"].apply(lambda x : x.replace(" ... "," pointdesuspension "))
df["text"] = df["text"].apply(lambda x : x.replace(" .. "," pointdesuspension "))
df["text"] = df["text"].apply(lambda x : x.replace(" .... "," pointdesuspension "))


In [ ]:
#truc à garder ou à tester :
# les chiffres
# les ! et ?
# faire gaffe aux ', par exemple aren't ou didn't, qui expriment une négation
# http://twitpic
# http://bit
# http://tinyurl
# http://plurk
# http://www
# http://blip
# &lt;3 qui vaut <3
# :) ;) :/ mais aussi : ) ou :'(
# &amp &gt et &quot qui ont l'air d'être des infos html
# w/o without
# w/ with
# b/c because
# ... .. et .... 


In [ ]:
# Find which words are upper to remove ones whithout info ('I', ...)
#df["text"][0] = "I AAA"
irrelevant_words = ['I']
#df["redondant"] = df["text"].apply(lambda x: True if "I" in x.split() else False)
x = df["text"]
for word in irrelevant_words:
  x = x.str.replace(word,'')

# Check if there is a UPPERCASE word in the series
df["upper"] = x.str.split()
df["upper"] = df["upper"].apply(lambda x: [i.isupper() for i in x])
df["upper"] = df["upper"].apply(lambda x: True if True in x else False)
#df["upper"].head()
confusion_matrix = pd.crosstab(df["target"], df["upper"], rownames=['target'], colnames=['is any upper'])
confusion_matrix/df.shape[0]*100

In [6]:
#------------------------Eliminate stop words-------------------------------
# Put everything to lower
df["text"]=df["text"].str.lower()
# remove stop_words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
irrelevant_words = stopwords.words('english')
irrelevant_words.append("i\'m")
df['text'] = df["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (irrelevant_words)]))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [42]:
#------------------------Data words Frequency-------------------------------
treshhold=0.1
dict_frequency=dict(tokenizer.word_counts)
col=pd.Series(dict_frequency, name='freq')
df_freq=pd.DataFrame(col.sort_values(ascending=False))
df_freq.reset_index(drop=False)
df_freq=df_freq[df_freq["freq"]>treshhold]
df_freq['freq']=df_freq['freq']/df_freq["freq"].sum()*100
#Verification
#df_freq["freq"].sum()

100.00000000000003

In [66]:
print(df_freq.head(vocab_size).to_string())

#### With all data

In [58]:
vocab_size = 1000
max_len = max(len(train_padded[0]))

sentences = df['text']
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<00V>", filters='0123456789!"#$%&()*+,-./:;<=>?@[\\]^_{|}~\t\n')
tokenizer.fit_on_texts(sentences)
word_index=tokenizer.word_index
sequences=tokenizer.texts_to_sequences(sentences)
padded=pad_sequences(sequences,padding="post")

#### Spliting data

In [ ]:
vocab_size = 1000
max_len = max(len(train_padded[0]))
sentences = df['text']
# Split Training and Testing
X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df["target"], test_size=0.33, random_state=42)
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<00V>")
tokenizer.fit_on_texts(X_train)
word_index=tokenizer.word_index

train_sequences=tokenizer.texts_to_sequences(X_train)
train_padded=pad_sequences(train_sequences,padding="post")

test_sequences=tokenizer.texts_to_sequences(X_test)
test_padded=pad_sequences(test_sequences,padding="post")

#### Model and Training

In [ ]:
embedding_dim = 16
model=tf.keras.Sequential(
    [
     tf.keras.layers.Embedding(vocab_size,embedding_dim),
     tf.keras.layers.GlobalAveragePooling1D(),
     tf.keras.layers.Dense(24,activation="relu"),
     tf.keras.layers.Dense(1,activation="sigmoid")
    ]
    )
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model.fit(train_padded,y_train,epochs=10,batch_size=100,validation_data=(test_padded,y_test))

#### LSTM

In [ ]:
my_model = model.my_model(max_words=vocab_size, max_len=228, embedd_size=vocab_size)

my_model.summary()

my_model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])


In [ ]:
my_model.fit(train_padded, y_train, epochs=10, batch_size=256, validation_data=(test_padded,y_test))
